In [40]:
import fasttext
import pandas as pd

In [4]:
model = fasttext.load_model('models/crawl-300d-2M-subword.bin')

In [39]:
model.get_nearest_neighbors('i7')

[(0.8886364102363586, 'i5'),
 (0.7744020223617554, 'i7-2600'),
 (0.7599759697914124, 'I7'),
 (0.7581802010536194, 'i7-2600k'),
 (0.7488381862640381, 'i5-2500k'),
 (0.7483259439468384, 'i7-3770k'),
 (0.7482477426528931, 'i7-950'),
 (0.7459214329719543, 'i7-3770'),
 (0.7458125352859497, 'i5-2500'),
 (0.7451770901679993, 'i7-4770')]

In [45]:
df = pd.read_json('data/computers_gs.json.gz', compression='gzip', lines=True)

In [46]:
print(df)

       id_left              category_left  cluster_id_left  \
0       581109  Computers_and_Accessories          1324529   
1      3083228  Computers_and_Accessories           640007   
2      5942105  Computers_and_Accessories          1464841   
3      1282014  Computers_and_Accessories           173820   
4      7969280  Computers_and_Accessories           224463   
...        ...                        ...              ...   
1095  10596400  Computers_and_Accessories         12508878   
1096  10186131  Computers_and_Accessories            38064   
1097   2803663                Video_Games           824793   
1098   8856662  Computers_and_Accessories           250672   
1099  11774848  Computers_and_Accessories           224463   

                                       identifiers_left  id_right  \
0     [{'/mpn': '[gvrx480g1gaming4gd]'}, {'/gtin13':...  16637861   
1     [{'/mpn': '[9hlf4lbdbe]'}, {'/gtin13': '[47187...   3424944   
2                              [{'/sku': '[mgeq2